# Встановлення залежностей

In [5]:
!pip install wfdb
!pip install py-ecg-detectors
!pip install sklearn
!pip install gensim
!pip install -U scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'c:\program files\python\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\program files\python\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\program files\python\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\program files\python\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\program files\python\python.exe -m pip install --upgrade pip' command.


In [6]:
import os
import wfdb
import numpy as np
import matplotlib.pyplot as plt

# Завантаження даних ЕКГ

In [7]:
if os.path.isdir("../mitdb"):
    print('You already have the data.')
else:
    wfdb.dl_database('../mitdb', 'mitdb')

You already have the data.


In [8]:
record = record = wfdb.rdsamp('../mitdb/100', sampto=500000)
record[1]
# record[0][:,0]
#annotation = wfdb.rdann('../mitdb/100','atr', sampfrom=50)
# record[1]
# record[0][:,0]
#sample = annotation.__dict__
#sample

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\mikle\\Documents\\0 KPI\\diploma\\word2vec-analysis\\classification\\mitdb\\100.hea'

# Виділення R піків

In [ ]:
from ecgdetectors import Detectors
fs = 250
detectors = Detectors(fs)
unfiltered_ecg = record[0][:, 1]
r_peaks = detectors.engzee_detector(unfiltered_ecg)
# Amendment for first peak
r_peaks[0] += 20

In [ ]:
r_peaks_split = r_peaks[:11]
plt.figure(figsize=(24,6))
plt.plot(unfiltered_ecg[:3100])
plt.plot(r_peaks_split, unfiltered_ecg[r_peaks_split], 'ro')
plt.show()

# Розподіл ЕКГ сигналу на серцебиття

In [ ]:
# Find heartbeat lenght
distances = []
for i in range(len(r_peaks)):
  if i + 1 < len(r_peaks):
    distances.append(r_peaks[i + 1] - r_peaks[i])
distances = np.array(distances)
heart_beat_len = int(distances.mean())
heart_beat_len

In [ ]:
# Beat EGC signal
beats = []
for i in range(len(r_peaks)):
  if i != 0 :
    r_index = r_peaks[i]
    retreat = heart_beat_len // 2
    beats.append(unfiltered_ecg[r_index - retreat:r_index + retreat])
len(beats)

# Візуалізація серцебиттів

In [ ]:
ig, ax = plt.subplots(nrows=10, figsize=(3,36))
plots = []
for beat in beats:
  plots.append(beat)



i = 0 
for plot in plots:
  if i < 10:
    ax[i].plot(plot)
    ax[i].set_ylabel('Heartbeat {}'.format(i + 1))
    ax[i].set_xlabel('Datapoints')
  i += 1

# Виділення хвиль в кожному серцебитті

In [ ]:
p_waves = []
qrs_waves = []
t_waves = []
for j in range(len(beats)):
  if j + 1 != len(beats):
    p_waves.append(beats[j][:retreat - 15])
    qrs_waves.append(beats[j][retreat - 15:retreat + 15])
    t_waves.append(beats[j][retreat + 15:])

print(len(t_waves[0]))
print(len(p_waves))

# Кластеризація виділених хвиль


In [ ]:
# P and T waves clustering (20 clusters)
from sklearn.cluster import KMeans
pt_waves = np.array(p_waves + t_waves)
print(pt_waves.shape)
kmeans = KMeans(init='k-means++', n_clusters=300, n_init=10)
kmeans.fit(pt_waves)
predict_pt = kmeans.predict(pt_waves)
predict_pt.shape

In [ ]:
# QRS waves clustering (6 clusters)
qrs_waves = np.array(qrs_waves)
kmeans_1 = KMeans(init='k-means++', n_clusters=40, n_init=10)
kmeans_1.fit(qrs_waves)
predict_qrs = kmeans_1.predict(qrs_waves)
predict_qrs.shape

# Присвоювання символів хвилі кожного кластеру

In [ ]:
alphabet_for_pt = {0:'a', 1:'b', 2:'c', 3:'d', 4:'e', 5:'f', 6:'g', 7:'h', 8:'i',
                   9:'j', 10:'k', 11:'l', 12:'m', 13:'n', 14:'o', 15:'p', 16:'q', 17:'r',
                   18:'s', 19:'t'}
alphabet_for_qrs = {0:'u', 1:'v', 2:'w', 3:'x', 4:'y', 5:'z'}

def get_symbol_pt(x):
  return alphabet_for_pt[x]

def get_symbol_qrs(x):
  return alphabet_for_qrs[x]

vfunc = np.vectorize(get_symbol_pt)
vfunc_2 = np.vectorize(get_symbol_qrs)

# predict_pt_letters = vfunc(predict_pt)
# predict_qrs_letters = vfunc_2(predict_qrs)


In [ ]:
#len(predict_qrs_letters)

In [ ]:
# words = []
# for i in range(len(predict_qrs_letters)):
#   word = ''
#   word += predict_pt_letters[i]
#   word += predict_qrs_letters[i]
#   word += predict_pt_letters[i + len(predict_qrs_letters)//2]
#   words.append(word)
#
# words

# Word2Vec model

In [ ]:
# from gensim.models import Word2Vec
# word2vec = Word2Vec([words,], min_count=0)

In [ ]:
# vocabulary = word2vec.wv.key_to_index
# vocabulary

In [ ]:
#sim = word2vec.wv.most_similar('jwl')
#sim

In [ ]:
#word2vec.wv['jwl']

In [ ]:
# from sklearn.datasets import make_blobs
# X, y_true = make_blobs(n_samples=300, centers=4,
#                        cluster_std=0.60, random_state=0)
# from sklearn.cluster import KMeans
# kmeansS = KMeans(n_clusters=4)
# kmeansS.fit(X)
# kmeansS.cluster_centers_.shape
#
# np.matrix([1, 2, 3])


In [ ]:
reversed_alphabet_for_pt = {}
for k,v in alphabet_for_pt.items():
  reversed_alphabet_for_pt[v] = k

reversed_alphabet_for_qrs = {}
for k,v in alphabet_for_qrs.items():
  reversed_alphabet_for_qrs[v] = k

pr_centers = kmeans.cluster_centers_
qrs_centers = kmeans_1.cluster_centers_
# KMeans Original data: 3484x128
print("pr_centers.shape:")
print(pr_centers.shape)
print("qrs_centers.shape:")
print(qrs_centers.shape)

def get_pr_from_symbol(x):
  return  pr_centers[x]

def get_qrs_from_symbol(x):
  return qrs_centers[x]

reverse_vfunc = np.vectorize(get_pr_from_symbol, signature='()->(n)')
reverse_vfunc_2 = np.vectorize(get_qrs_from_symbol, signature='()->(n)')

predict_pt_original = reverse_vfunc(predict_pt)
predict_qrs_original = reverse_vfunc_2(predict_qrs)
print("predict_pt_original.shape:")
print(predict_pt_original.shape)
print("predict_qrs_original.shape:")
print(predict_qrs_original.shape)

splited_pt = np.split(predict_pt_original, 2)
predict_t_original = splited_pt[0]
predict_p_original = splited_pt[1]
print(str(predict_pt_original.shape) + " = " + str(predict_t_original.shape) + " + " + str(predict_p_original.shape))

restored_beats = np.concatenate((predict_t_original, predict_qrs_original, predict_p_original), axis=1)
print("restored_beats.shape = " + str(restored_beats.shape))

In [ ]:
print("Original:")
print(beats[0].shape)
print("Restored:")
print(restored_beats[0].shape)

fig, ax = plt.subplots(nrows=10, ncols=2, figsize=(6,36))
restored_plots = []
for beat in restored_beats:
  restored_plots.append(beat)

original_plots = []
for beat in beats:
  original_plots.append(beat)

for i in range(10):
  ax1 = ax[i][0]
  ax2 = ax[i][1]
  ax1.plot(restored_plots[i])
  ax1.set_ylabel('Heartbeat {}'.format(i + 1))
  ax1.set_xlabel('Datapoints')

  ax2.plot(original_plots[i])
  ax2.set_ylabel('Heartbeat {}'.format(i + 1))
  ax2.set_xlabel('Datapoints')

plt.savefig("comparing.png", transparent=False)

In [ ]:
from sklearn.metrics import mean_absolute_error
for i in range(10):
  rmspe = np.sqrt(np.mean(np.square(((beats[i] - restored_beats[i]) / beats[i])), axis=0))
  print(rmspe)


# len(words)
# len(vocabulary)
# kmeans.cluster_centers_.shape
# kmeans_1.cluster_centers_.shape
# #kmeans.score(predict_pt)
# predict_pt
# #kmeans.cluster_centers_[0]


In [ ]:
r_peaks_split = r_peaks[:11]
plt.figure(figsize=(24,6))
original_full = np.concatenate(beats)[:3100]
plt.plot(original_full)
restored_full = np.concatenate(restored_beats)[:3100]
plt.plot(restored_full)
plt.savefig("comparing_original.png", transparent=False)

In [1]:
np.sqrt(np.mean(np.square(((original_full - restored_full) / original_full)), axis=0))

NameError: name 'np' is not defined

In [2]:
%history -g -f random_forest.ipynb

 1/1:
import numpy as np
import matplotlib.pyplot as plt
 1/2:
import numpy as np
import matplotlib.pyplot as plt
 1/3:
import numpy as np
import matplotlib.pyplot as plt
 1/4:
import numpy as np
import matplotlib.pyplot as plt
 1/5:
import numpy as np
import matplotlib.pyplot as plt
 1/6:
def sigmoid(x):
    return 1/(1+e(-x))

x = np.linspace(-5, 5, 100)
y = sigmoid(x)

plt.plot(x, y)
plt.ylim(0, 1)
plt.show()
 1/7:
import numpy as np
import matplotlib.pyplot as plt
 1/8:
def sigmoid(x):
    return 1/(1+math.e(-x))

x = np.linspace(-5, 5, 100)
y = sigmoid(x)

plt.plot(x, y)
plt.ylim(0, 1)
plt.show()
 1/9:
def sigmoid(x):
    return 1 / (1 + math.e(-x))

x = np.linspace(-5, 5, 100)
y = sigmoid(x)

plt.plot(x, y)
plt.ylim(0, 1)
plt.show()
1/10:
def sigmoid(x):
    return 1 / (1 + math.e(-x))

x = np.linspace(-5, 5, 100)
y = sigmoid(x)

plt.plot(x, y)
plt.ylim(0, 1)
plt.show()
1/11:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

x = np.linspace(-5, 5, 100)
y = sigmoid(x)

plt.plot(x